In [5]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import BasicAer, execute
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit.extensions.standard import HGate, SGate, SdgGate, XGate
import numpy as np
from qiskit.quantum_info.states.measures import state_fidelity
import copy
import itertools
import cutter

def simulate_circ(circ, simulator='statevector_simulator'):
    backend = BasicAer.get_backend(simulator)
    job = execute(circ, backend=backend)
    result = job.result()
    outputstate = result.get_statevector(circ)
    outputprob = [np.power(abs(x),2) for x in outputstate]
    return outputstate

q = QuantumRegister(3,'q')
original_circ = QuantumCircuit(q)
original_circ.h(0)
original_circ.t(1)
original_circ.cx(0,1)
original_circ.cx(1,2)
original_circ.cx(0,1)
original_circ.y(2)
original_circ_sv = simulate_circ(original_circ)
print('original circuit:')
print(original_circ)
print('uncut circuit state vector:', original_circ_sv)
print('*'*100)

positions = [(q[0],1),(q[1],1)]
clusters, complete_path_map, K, d = cutter.cut_circuit(original_circ, positions)

print('Complete path map:')
[print(x, ': ',complete_path_map[x]) for x in complete_path_map]
for idx, x in enumerate(clusters):
    print('cluster %d circuit:'%idx)
    print(x)
print('*'*50)

original circuit:
        ┌───┐               
q_0: |0>┤ H ├──■─────────■──
        ├───┤┌─┴─┐     ┌─┴─┐
q_1: |0>┤ T ├┤ X ├──■──┤ X ├
        └───┘└───┘┌─┴─┐├───┤
q_2: |0>──────────┤ X ├┤ Y ├
                  └───┘└───┘
uncut circuit state vector: [0.+0.j         0.-0.70710678j 0.+0.j         0.+0.j
 0.+0.70710678j 0.+0.j         0.+0.j         0.+0.j        ]
****************************************************************************************************
Complete path map:
Qubit(QuantumRegister(3, 'q'), 0) :  [(0, Qubit(QuantumRegister(2, 'q'), 0)), (1, Qubit(QuantumRegister(2, 'cutQ'), 1))]
Qubit(QuantumRegister(3, 'q'), 1) :  [(0, Qubit(QuantumRegister(2, 'q'), 1)), (1, Qubit(QuantumRegister(2, 'cutQ'), 0))]
Qubit(QuantumRegister(3, 'q'), 2) :  [(1, Qubit(QuantumRegister(1, 'q'), 0))]
cluster 0 circuit:
        ┌───┐     
q_0: |0>┤ H ├──■──
        ├───┤┌─┴─┐
q_1: |0>┤ T ├┤ X ├
        └───┘└───┘
cluster 1 circuit:
           ┌───┐┌───┐
   q_0: |0>┤ X ├┤ Y ├
           └─┬─┘├──

In [6]:
cluster_0_meas = simulate_circ(clusters[0])
all_s = list(itertools.product(range(0,2),repeat=len(positions)))
recombine = [0 for x in original_circ_sv]
for idx, s in enumerate(all_s):
    print('s = ', s)
    print(clusters[0])
    cluster_0_sv = cluster_0_meas[idx]
    print('cluster_0_sv = ', cluster_0_sv)
    
    cluster_1_copy = copy.deepcopy(clusters[1])
    cluster_1_dag = circuit_to_dag(cluster_1_copy)
    
    if s[0] == 1:
        cluster_1_dag.apply_operation_front(op=XGate(),qargs=[cluster_1_copy.qubits[1]],cargs=[])
    if s[1] == 1:
        cluster_1_dag.apply_operation_front(op=XGate(),qargs=[cluster_1_copy.qubits[2]],cargs=[])
    cluster_1_circ = dag_to_circuit(cluster_1_dag)
    cluster_1_sv = simulate_circ(cluster_1_circ)
    cluster_1_sv_ordered = [0 for x in cluster_1_sv]
    cluster_1_sv_ordered[0] = cluster_1_sv[0]
    cluster_1_sv_ordered[1] = cluster_1_sv[4]
    cluster_1_sv_ordered[2] = cluster_1_sv[2]
    cluster_1_sv_ordered[3] = cluster_1_sv[6]
    cluster_1_sv_ordered[4] = cluster_1_sv[1]
    cluster_1_sv_ordered[5] = cluster_1_sv[5]
    cluster_1_sv_ordered[6] = cluster_1_sv[3]
    cluster_1_sv_ordered[7] = cluster_1_sv[7]
    print(cluster_1_circ)
    print('cluster_1_sv = ', cluster_1_sv_ordered)
    recombine_term = np.kron(cluster_0_sv, cluster_1_sv_ordered)
    recombine += recombine_term
    print('-'*50)

fid = state_fidelity(recombine, original_circ_sv)
print('reconstruction fidelity =', fid)

s =  (0, 0)
        ┌───┐     
q_0: |0>┤ H ├──■──
        ├───┤┌─┴─┐
q_1: |0>┤ T ├┤ X ├
        └───┘└───┘
cluster_0_sv =  (0.7071067811865476+0j)
           ┌───┐┌───┐
   q_0: |0>┤ X ├┤ Y ├
           └─┬─┘├───┤
cutQ_0: |0>──■──┤ X ├
                └─┬─┘
cutQ_1: |0>───────■──
                     
cluster_1_sv =  [0j, 0j, 0j, 0j, 1j, 0j, 0j, 0j]
--------------------------------------------------
s =  (0, 1)
        ┌───┐     
q_0: |0>┤ H ├──■──
        ├───┤┌─┴─┐
q_1: |0>┤ T ├┤ X ├
        └───┘└───┘
cluster_0_sv =  0j
           ┌───┐┌───┐
   q_0: |0>┤ X ├┤ Y ├
           └─┬─┘├───┤
cutQ_0: |0>──■──┤ X ├
           ┌───┐└─┬─┘
cutQ_1: |0>┤ X ├──■──
           └───┘     
cluster_1_sv =  [0j, 0j, 0j, 0j, 0j, 0j, 0j, 1j]
--------------------------------------------------
s =  (1, 0)
        ┌───┐     
q_0: |0>┤ H ├──■──
        ├───┤┌─┴─┐
q_1: |0>┤ T ├┤ X ├
        └───┘└───┘
cluster_0_sv =  0j
                ┌───┐┌───┐
   q_0: |0>─────┤ X ├┤ Y ├
           ┌───┐└─┬─┘├───┤
cutQ_0: |0>┤